In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#pip install pandas openpyxl


In [ ]:
data = pd.read_excel('/kaggle/input/ecg-ds-health-care/DS-Healthcare.xlsx',header = 0)


In [ ]:
data.head(20)

In [ ]:

new_column_names = data.iloc[0]  
data.columns = new_column_names  
data = data[1:]  


In [ ]:
data.head(10)

In [ ]:
data.tail(100)

In [ ]:
data.Type.value_counts()

In [ ]:
data['Type'] = data['Type'].map({'arrhythmic': 'unhealthy', 'ischemic': 'unhealthy', 'healthy': 'healthy'})




In [ ]:
data.head()

In [ ]:
data["Type"].value_counts()

In [ ]:
# Assuming your DataFrame is named 'data'
null_count = data.isnull().sum()

# Display the count of null values for each column
print(null_count)


In [ ]:
total_null_count = null_count.sum()
print(f'Total number of null values: {total_null_count}')


In [ ]:
!pip install -U imbalanced-learn


In [ ]:
data= data.drop('Subject', axis=1)
data= data.drop('Cycle', axis=1)      #added by Rafsan
data= data.drop('Time', axis=1)      # added by Rafsan

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming your DataFrame is named 'data'
label_encoder = LabelEncoder()

# Apply label encoding to the 'Type' column
data['Type'] = label_encoder.fit_transform(data['Type'])


In [ ]:
data.Type.value_counts()

__________________________________________________________________________________________________________________________________________

# New Code Added

In [ ]:
data.info()

In [ ]:
data= data.apply(pd.to_numeric, errors='coerce') #added by rafsan

In [ ]:
data.info()

In [156]:
X = data.loc[:,data.columns!='Type']
y = data['Type']

In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [158]:
X_train = tf.convert_to_tensor(X_train.values.astype(np.float64))
X_test = tf.convert_to_tensor(X_test.values.astype(np.float64))
y_train = tf.convert_to_tensor(y_train.values.astype(np.float64))
y_test = tf.convert_to_tensor(y_test.values.astype(np.float64))

In [159]:
X_train.shape, y_train.shape

(TensorShape([838859, 9]), TensorShape([838859]))

### LSTM

In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, Flatten, Dense
# Define the LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], 1)))
#model.add(Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.15)

### RNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(50, input_shape=(X_train.shape[1],1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model (Assuming X_train and y_train are your training data)
model.fit(X_train, y_train, epochs=5, batch_size=32)


### CNN from (previous notebook from shoaib)

In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2)


________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Assuming your DataFrame is named 'data'
# Assuming 'Type' is the column with the class labels

# Drop non-numeric columns or encode them if needed
data_numeric = data.select_dtypes(include=['number'])

# Create feature matrix X and target variable y
X = data_numeric.drop('Type', axis=1).values
y = data_numeric['Type'].values

# Convert labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert numerical labels to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Print input size
print("Input size:", X_train.size())

# Define the MLP model in PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
input_size = X_train.size(1)  # Adjusted input size
hidden_size = 64  # You can adjust the hidden size as needed
num_classes = len(label_encoder.classes_)
model = MLPModel(input_size, hidden_size, num_classes)

# Print model architecture
print(model)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    _, y_pred_labels = torch.max(y_pred, 1)
    y_true_labels = y_test

# Convert predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred_labels.numpy())
y_true_original = label_encoder.inverse_transform(y_true_labels.numpy())

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_true_original, y_pred_original)
report = classification_report(y_true_original, y_pred_original)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


In [ ]:
data.head()

In [ ]:
%time 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Assuming your DataFrame is named 'data'
# Assuming 'Type' is the column with the class labels

# Drop non-numeric columns or encode them if needed
data_numeric = data.select_dtypes(include=['number'])

# Create feature matrix X and target variable y
X = data_numeric.drop('Type', axis=1)
y = data_numeric['Type']

# Convert labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert numerical labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Set the maximum window size based on available data
max_window_size = len(X)

# Decrease the window size until it fits the data
input_window_size = max_window_size
while input_window_size > 1:
    X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
    
    X_train_reshaped = np.array([X_train.iloc[i:i+input_window_size].values for i in range(len(X_train)-input_window_size+1)])
    X_test_reshaped = np.array([X_test.iloc[i:i+input_window_size].values for i in range(len(X_test)-input_window_size+1)])
    
    if X_train_reshaped.size > 0 and X_test_reshaped.size > 0:
        break
    
    input_window_size -= 1
print("Printed")

# Check if the adjusted window size is suitable
if input_window_size <= 1:
    raise ValueError("No suitable window size found. Adjust the window size or provide more data.")

# Print the adjusted window size
print(f"Adjusted input window size: {input_window_size}")

# Reshape the input data to have one channel
X_train_reshaped = X_train_reshaped.reshape((X_train_reshaped.shape[0], X_train_reshaped.shape[1], 1))
X_test_reshaped = X_test_reshaped.reshape((X_test_reshaped.shape[0], X_test_reshaped.shape[1], 1))

# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
import time

# ... (your existing code)

# Record the start time
start_time = time.time()

# Train the model
print("Printed")
model.fit(X_train_reshaped, y_train, epochs=2, batch_size=32, validation_split=0.2)
end_time = time.time()

# Calculate the time taken
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

# Evaluate the model on the test set
y_pred = model.predict(X_test_reshaped)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Convert predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred_labels)
y_true_original = label_encoder.inverse_transform(y_true_labels)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_true_original, y_pred_original)
report = classification_report(y_true_original, y_pred_original)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)
